In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_files
import re
import glob
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
import sys
from functools import partial
from zipfile import ZipFile

In [2]:
data = []
with ZipFile("data.zip") as z:
    for fname in z.namelist():
        if not fname.endswith('.txt') or not fname.startswith('data'):
            continue
        content = z.read(fname).decode('utf8')
        label, idx = fname[5:-4].split('-', 1)
        data.append((idx,label,content,))
df_train = pd.DataFrame(data, columns=("id", "label", "content"))
df_train.sample(10)

,id,label,content
1559,12650,employment,##### \n\nExhibit 10.1 \n\n\n\nTHIS EMPLOYMENT...
1492,11980,employment,Exhibit 10.23\n\n\n\nThis Employment Agreement...
996,07020,employment,Exhibit 10.1 \n\nEXECUTION COPY \n\n\n\nThis E...
2019,17250,employment,##### \n\n\n\nThis Agreement is made effective...
4377,01340,SPA,\n\n\n\n\n \n\n\nThis STOCK PURCHASE AGREEM...
1538,12440,employment,\n\n\nExhibit 10.1 Employment Agreement of M...
653,03590,employment,Exhibit 99.3\n\nConformed Executed Version\n\n...
292,42420,bylaws,Exhibit 3.2 \n \n\n \nOF \n \nMERCARI COM...
536,02420,credit,Exhibit 4.12\n\n \n\n\n\n\nAND\n\nSTOCK PURCH...
457,44070,bylaws,EXHIBIT 3(i)\n\n\n\n \n\n\nOF\n\n \n\n\nHNI ...


In [3]:
# Read the train data
#train_data = load_files(r'data', categories=['bylaws', 'credit', 'employment', 'rra', 'rsu', 'spa'])

In [4]:
#X, y = train_data.data, train_data.target_names
#print(y[0])

X = df_train['content'].to_list()

y_data = df_train['label'].to_list()


In [ ]:
# Txt preprocessing
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_data = vectorizer.fit_transform(documents).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X_data = tfidfconverter.fit_transform(X_data).toarray()

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


SVM_classifier = SVC(random_state=0)
SVM_classifier.fit(X_train, y_train) 

y_pred = SVM_classifier.predict(X_test)


In [ ]:
print(df_train.label.hist())

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
# Read the test data

data = []
with ZipFile("test.zip") as z:
    for fname in z.namelist():
        content = z.read(fname).decode('utf8')
        if not fname.endswith('.txt') or not fname.startswith('test'):
            continue
        idx = fname[5:-4]
        data.append((idx,content,))
df_test = pd.DataFrame(data, columns=("id", "content"))
df_test.sample(10)

X_test_data = df_test['content'].to_list()


In [ ]:
print(df_test.head())

In [ ]:
# Txt preprocessing
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X_test_data)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X_test_data[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_test_data = vectorizer.fit_transform(documents).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X_test_data = tfidfconverter.fit_transform(X_test_data).toarray()

In [ ]:
y_pred = SVM_classifier.predict(X_test_data)


In [ ]:
df_test['label'] = y_pred
print(df_test.label.hist())

In [ ]:
import sys
from functools import partial
from zipfile import ZipFile
import pandas as pd
sys.path.append('../src')
%matplotlib inline

In [ ]:
from urllib import request, parse
import json, random

def submit(name,company,email,phone,user, test_data):
    """Submits a test data set to the pydata server, and returns accuracy"""
    try:
        test_data = {str(k).lower().replace(".txt", ""):str(v).lower().strip() for k,v in test_data.items()}
        data = parse.urlencode({"user":user, "submission": json.dumps(test_data), "name":name,"company":company,"phone":phone,"email":email}).encode()
        req =  request.Request("https://goren.ml/uattcontract/index.php?rand="+str(random.randint(167,1000000)), data=data)
        resp = request.urlopen(req)
        return float(resp.read().decode("utf8"))
    except Exception as e:
        print (str(e))
        return None

In [ ]:
# If you're having problem importing this file, copy and paste the contents of "uatt.py" in this cell instead
# of the import statement
#from uatt import submit

my_submit = partial(submit, "Nir Cohen", "IAI", "nir.cohen13@gmail.com", "0546840888")

In [ ]:
submission = dict(df_test[["id","label"]].values)
#submission name must be unique
submission_name = "Nir_Cohen_1"
my_submit(submission_name, submission)

In [ ]:
import random, string
from IPython.display import HTML
rand_str = lambda: "".join(random.sample(string.ascii_letters,7))
HTML('<a href="https://goren.ml/uattcontract/?{k}={v}" target="_blank">Go to Leaderboard</a>'.format(k=rand_str(),v=rand_str()))

In [ ]:
# Save model
#with open('text_classifier', 'wb') as picklefile:
#    pickle.dump(classifier,picklefile)

In [ ]:
# Read the test data

#data = []
#with ZipFile("test.zip") as z:
#    for fname in z.namelist():
#        content = z.read(fname).decode('utf8')
#        if not fname.endswith('.txt') or not fname.startswith('test'):
#            continue
#        idx = fname[5:-4]
#        data.append((idx,content,))
#df_test = pd.DataFrame(data, columns=("id", "content"))
#df_test.sample(10)

#X_test = df_test['content'].to_list()


#path = os.path.dirname('C:\Users\nirco\contract_classification\notebooks\test')
#csv_files = glob.glob(os.path.join(path, "*.csv"))

#csv_files = glob.glob('test')

#train_data = load_files(r'test')
#X_test = train_data.data

# List comprehension that loads of all the files
# X_test = [pd.read_csv(x) for x in path]

